<a href="https://colab.research.google.com/github/saanvi1211/LLM-Project/blob/main/Gpt_from_scratch_attempt5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**A transformer-based language model architecture**



In [ ]:
import chardet

# Function to detect file encoding
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        rawdata = f.read()
    result = chardet.detect(rawdata)
    return result['encoding']

# Detect encoding of the file
file_encoding = detect_encoding('mergedfile.txt')
print("Detected encoding:", file_encoding)

# Read the input text using detected encoding
with open('mergedfile.txt', 'r', encoding=file_encoding) as f:
    text = f.read()


Detected encoding: MacRoman


In [ ]:
with open('mergedfile.txt', 'r', encoding='macRoman') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  641426


In [ ]:

print(text[:1000])

PART I
THE UNION AND ITS TERRITORY
1. Name and territory of the Union.‚Äî(1) India, that is Bharat, shall be
a Union of States.
(2) The States and the territories thereof shall be as specified in the First
Schedule.
(3) The territory of India shall comprise‚Äî
(a) the territories of the States;
(b) the Union territories specified in the First Schedule; and
(c) such other territories as may be acquired.
2. Admission or establishment of new States.‚ÄîParliament may by law
admit into the Union, or establish, new States on such terms and conditions as i
t
thinks fit.
2A. [Sikkim to be associated with the Union.] Rep. by the Constitution
(Thirty- sixth Amendment) Act, 1975, s. 5 (w.e.f. 26-4-1975).
3. Formation of new States and alteration of areas, boundaries or
names of existing States.‚ÄîParliament may by law‚Äî
(a)form a new State by separation of territory from any State or by
uniting two or more States or parts of States or by uniting any territory to
a part of any State;
(b) increase

In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 "'()*,-.0123456789:;ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz¶ÄÔìîòôùú‚™
89


In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[59, 60, 60, 2, 71, 59, 56, 69, 56]
hii there


In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([641426]) torch.int64
tensor([38, 23, 40, 42,  2, 31,  0, 42, 30, 27,  2, 43, 36, 31, 37, 36,  2, 23,
        36, 26,  2, 31, 42, 41,  2, 42, 27, 40, 40, 31, 42, 37, 40, 47,  0, 12,
        10,  2, 36, 52, 64, 56,  2, 52, 65, 55,  2, 71, 56, 69, 69, 60, 71, 66,
        69, 76,  2, 66, 57,  2, 71, 59, 56,  2, 43, 65, 60, 66, 65, 10, 87, 79,
        82,  5, 12,  6,  2, 31, 65, 55, 60, 52,  8,  2, 71, 59, 52, 71,  2, 60,
        70,  2, 24, 59, 52, 69, 52, 71,  8,  2, 70, 59, 52, 63, 63,  2, 53, 56,
         0, 52,  2, 43, 65, 60, 66, 65,  2, 66, 57,  2, 41, 71, 52, 71, 56, 70,
        10,  0,  5, 13,  6,  2, 42, 59, 56,  2, 41, 71, 52, 71, 56, 70,  2, 52,
        65, 55,  2, 71, 59, 56,  2, 71, 56, 69, 69, 60, 71, 66, 69, 60, 56, 70,
         2, 71, 59, 56, 69, 56, 66, 57,  2, 70, 59, 52, 63, 63,  2, 53, 56,  2,
        52, 70,  2, 70, 67, 56, 54, 60, 57, 60, 56, 55,  2, 60, 65,  2, 71, 59,
        56,  2, 28, 60, 69, 70, 71,  0, 41, 54, 59, 56, 55, 72, 63, 56, 10,  0,
       

In [ ]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([38, 23, 40, 42,  2, 31,  0, 42, 30])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([38]) the target: 23
when input is tensor([38, 23]) the target: 40
when input is tensor([38, 23, 40]) the target: 42
when input is tensor([38, 23, 40, 42]) the target: 2
when input is tensor([38, 23, 40, 42,  2]) the target: 31
when input is tensor([38, 23, 40, 42,  2, 31]) the target: 0
when input is tensor([38, 23, 40, 42,  2, 31,  0]) the target: 42
when input is tensor([38, 23, 40, 42,  2, 31,  0, 42]) the target: 30


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 2, 54, 66, 65, 71, 60, 65, 72],
        [73, 56, 55,  2, 57, 66, 69,  2],
        [71, 69, 52, 65, 70, 57, 56, 69],
        [52, 63, 63,  2, 64, 52, 61, 66]])
targets:
torch.Size([4, 8])
tensor([[54, 66, 65, 71, 60, 65, 72, 56],
        [56, 55,  2, 57, 66, 69,  2, 41],
        [69, 52, 65, 70, 57, 56, 69,  2],
        [63, 63,  2, 64, 52, 61, 66, 69]])
----
when input is [2] the target: 54
when input is [2, 54] the target: 66
when input is [2, 54, 66] the target: 65
when input is [2, 54, 66, 65] the target: 71
when input is [2, 54, 66, 65, 71] the target: 60
when input is [2, 54, 66, 65, 71, 60] the target: 65
when input is [2, 54, 66, 65, 71, 60, 65] the target: 72
when input is [2, 54, 66, 65, 71, 60, 65, 72] the target: 56
when input is [73] the target: 56
when input is [73, 56] the target: 55
when input is [73, 56, 55] the target: 2
when input is [73, 56, 55, 2] the target: 57
when input is [73, 56, 55, 2, 57] the target: 66
when input is [73, 

In [ ]:
print(xb) # our input to the transformer

tensor([[ 2, 54, 66, 65, 71, 60, 65, 72],
        [73, 56, 55,  2, 57, 66, 69,  2],
        [71, 69, 52, 65, 70, 57, 56, 69],
        [52, 63, 63,  2, 64, 52, 61, 66]])


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 89])
tensor(4.8514, grad_fn=<NllLossBackward0>)

7rMu;-PUN¶î"EL:DAAc"UOì5),IoMM):HhMSO ÔúJK0QTzMyY8™9k"::_94Wmd¶QjZW‚Zz"JUaR1úÔqFAv™bù4™cÄn'I6M]cC2:R


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.757412910461426


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


IZEF, 6_XG,ù2bVJKiDUnôaA,uXìV9T.WXZCCV8n).EO ôWTA)*'0zH"cFW™ì 5T,5BUGH_™iC6lXlPw5wú3útôùòbUVCKOkAqEDypi[úev6K™Yr1h8ô¶î_™.B30, F6NùH0XUzHfÄ]DYDMEF)NôAr(.af_FReR;jIw3ùîc.BÄ4Mm1(I,ùôiDU6k"7'nO¶Y ù4nuVRF™Ry k[ôhSPn.ú:î)Mmvr4LJ-:Ix
cD "-Ô™6e(AvDkFUsOTìfkJHDY(81HL:eÔîEÔkn'ìOaPE¶D™)jbJk7™caJHoòV-NpSNH:A]îpH - HkX*N0ZLGg]f;yQ:7gvA,(KBvQJp-KBùwPx.búW'a Ôu9™Wxio 25K"gd)CSdò4VKôCì;ÔXElxaNWU.WkK"J(tb3 0iDK"xùîfO9.pJU.‚BQvD00G_JPGújgCXbMZG3C:PtEezr3ni(ò0N4Zbù), OÄt
jv1oPB4SKRoòWreRùòyPU;A;Z4m.mI',ùHúFK


In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var()

tensor(1.0449)

In [ ]:
q.var()

tensor(1.0700)

In [ ]:
wei.var()

tensor(1.0918)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
class LayerNorm1d:
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hyperparameters
batch_size = 16
block_size = 32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128
n_head = 4
n_layer = 4
dropout = 0.0

torch.manual_seed(1337)

# Open and read the file
with open('mergedfile.txt', 'r', encoding='MacRoman') as f:
    text = f.read()

# Define vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# Train and validation splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# Data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Define attention head module
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

# Define multi-head attention module
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

# Define feedforward module
class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

# Adjust Block module to include Layer Normalization after multi-head attention
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)  # Add Layer Normalization after multi-head attention
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# Define the language model
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# Instantiate the model
model = BigramLanguageModel()
m = model.to(device)

# Calculate the number of parameters
print(sum(p.numel() for p in m.parameters()) / 1e6, 'M parameters')

# Define the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Generate text from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=7000)[0].tolist()))


0.818777 M parameters
step 0: train loss 4.6787, val loss 4.6730
step 100: train loss 2.4254, val loss 2.7866
step 200: train loss 2.1815, val loss 2.5155
step 300: train loss 1.9397, val loss 2.4372
step 400: train loss 1.7751, val loss 2.3423
step 500: train loss 1.6281, val loss 2.1966
step 600: train loss 1.5551, val loss 2.1633
step 700: train loss 1.4786, val loss 2.0805
step 800: train loss 1.4450, val loss 2.0907
step 900: train loss 1.3774, val loss 2.0585
step 1000: train loss 1.3538, val loss 2.0570
step 1100: train loss 1.3288, val loss 1.9663
step 1200: train loss 1.2831, val loss 1.9611
step 1300: train loss 1.2740, val loss 1.9561
step 1400: train loss 1.2581, val loss 1.9366
step 1500: train loss 1.2356, val loss 1.9058
step 1600: train loss 1.2302, val loss 1.8990
step 1700: train loss 1.1972, val loss 1.8969
step 1800: train loss 1.1909, val loss 1.8293
step 1900: train loss 1.1833, val loss 1.8838
step 2000: train loss 1.1651, val loss 1.8367
step 2100: train loss 1.